In [10]:
eeve_file = open("./eeve_example.txt", "r", encoding="utf-8")
exaone_file = open("./exaone_example.txt", "r", encoding="utf-8")
Llama3_file = open("./Llama3_example.txt", "r", encoding="utf-8")

In [16]:
# 평가할 답변 샘플
answer_EEVE = eeve_file.read()
answer_LLAMA3 = Llama3_file.read()
answer_EXAONE = exaone_file.read()

In [19]:
# query 

user_query = "65세 여성 노인이 보건의료 관련해서 제주도에서 지원받을 수 있는 정책은 뭐가 있어?"

In [20]:
JUDGE_COMPARE_TEMPLATE = """너는 동일한 질문에 대해 세 개의 한국어 언어 모델이 생성한 응답을 매우 꼼꼼히 평가할 것이다. 공정하고 객관적인 판단을 위해 아래의 규칙을 철저히 준수한다.

# 기본 평가 기준
1. 질문에 대한 요구사항을 얼마나 잘 충족했는가?
2. 정책명, 금액, 조건 등 구체적인 정보가 포함되어 있는가?
3. 표현이 자연스럽고 이해하기 쉬운가?
4. 누락되었거나 아쉬운 정보는 없는가?

# 언어 요구사항
- 모델의 응답은 반드시 한국어로 작성되어야 하며, 그렇지 않을 경우 해당 응답은 0점 처리된다.
- 답변 길이의 차이는 평가에 영향을 미쳐서는 안 되며, 내용의 충실함에 집중한다.

# 평가 지침
- 각 모델의 응답을 정성적으로 분석한 뒤, 위 기준에 따라 상대적으로 가장 뛰어난 응답을 선정한다.
- 판단 이유를 간결하면서도 핵심적으로 기술한다.
- '모델 A', '모델 B', '모델 C' 중 하나를 선택하며, 반드시 이유와 함께 출력한다.

[질문]
{question}

[모델 A의 응답]
{answer_A}

[모델 B의 응답]
{answer_B}

[모델 C의 응답]
{answer_C}

# 출력 형식
선택: 모델 A / 모델 B / 모델 C
이유: (핵심 근거를 포함한 평가 내용. 3~5문장)"""

In [21]:
judge_prompt = JUDGE_COMPARE_TEMPLATE.format(
    question=user_query,
    answer_A=answer_EEVE,
    answer_B=answer_LLAMA3,
    answer_C=answer_EXAONE
)

In [22]:
from langchain_openai import ChatOpenAI

# 평가 LLM

llm_judge = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

In [25]:
from langchain_core.messages import HumanMessage

judge_response = llm_judge.invoke([HumanMessage(content=judge_prompt)])
print(judge_response.content)

선택: 모델 C
이유: 모델 C는 제주도에서 65세 여성 노인이 보건의료 관련해서 받을 수 있는 구체적인 지원 정책을 명확히 설명하고 있다. 정책명과 함께 지원 내용, 금액, 조건 등을 상세히 언급하여 질문에 대한 요구사항을 잘 충족하고 있다. 또한, 표현이 자연스럽고 이해하기 쉬워 정보 전달이 효과적이다. 반면, 모델 A와 모델 B는 구체적인 정보가 부족하거나 표현이 다소 모호하여 상대적으로 덜 충실한 답변을 제공하였다.


In [26]:
# 데이터 저장

with open("evaluation_result.txt", "w", encoding="utf-8") as f:
    f.write(judge_response.content)